In [1]:
#은닉 계층 하나를 위한 파라미터 생성 함수 정의
def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt
    
    pm_hidden = alloc_param_pair([input_cnt, hidden_cnt]) #출력 계층
    pm_output = alloc_param_pair([hidden_cnt, output_cnt]) #은닉 계층
    
def alloc_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape) #가중치 행렬 생성
    bias = np.zeros(shape[-1]) #편향 벡터 생성
    return {'w':weight, 'b':bias} #파이썬 사전 구조로 묶음

#은닉 계층 하나를 위한 순전파 함수 정의
def forward_neuralnet_hidden1(x): #위 함수에서 생성된 두쌍의 파라미터에 접근
    global pm_output, pm_hidden
    
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b']) #입력 x와 pm_hidden을 이용해 은닉 계층 출력인 hidden을 계산
    output = np.matmul(hidden, pm_output['w']) + pm_output['b'] #신경망의 최종 출력인 output계산
    
    return output, [x, hidden]

def relu(x): #선형 연산 결과에 적용
    return np.maximum(x, 0)

#은닉 계층 하나를 위한 역전파 함수 정의
def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    
    x, hidden = aux
    
    g_output_w_out = hidden.transpose()                      
    G_w_out = np.matmul(g_output_w_out, G_output)            
    G_b_out = np.sum(G_output, axis=0)                       

    g_output_hidden = pm_output['w'].transpose()             
    G_hidden = np.matmul(G_output, g_output_hidden)          

    pm_output['w'] -= LEARNING_RATE * G_w_out                
    pm_output['b'] -= LEARNING_RATE * G_b_out                
    
    G_hidden = G_hidden * relu_derv(hidden)
    
    g_hidden_w_hid = x.transpose()                           
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)            
    G_b_hid = np.sum(G_hidden, axis=0)                       
    
    pm_hidden['w'] -= LEARNING_RATE * G_w_hid                
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid                
    
def relu_derv(y):
    return np.sign(y)

#가변적 은닉 계층 구성을 위한 파라미터 생성 함수 정의
def init_model_hiddens():
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config #전역변수 hidden_config 리스트 통해 지정
    
    pm_hiddens = []
    prev_cnt = input_cnt #초기화 과정
    
    for hidden_cnt in hidden_config:
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt])) #생성된 파라미터 쌍들은 pm_hiddens 리스트 변수에 수십되어 전역변수로 전달
        prev_cnt = hidden_cnt #반복문 처리가 종료된 시점에 prev_cnt값은 마지막 은닉 계층의 출력 벡터 크기를 가지게 됨
    
    pm_output = alloc_param_pair([prev_cnt, output_cnt]) #출력 계층의 입력벡터 크기로 이용

#가변적 은닉 계층 구성을 위한 순전파 함수 정의    
def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens
    
    hidden = x #x로 초기화
    hiddens = [x] #hiddens 리스트를 두어 최초 입력인 x를 맨 앞에
    
    for pm_hidden in pm_hiddens: 
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b']) #x로 초기화한 후 반복 과정에서 은닉 계층 처리의 입력이자 출력으로 이용
        hiddens.append(hidden) #반복 각 단계에서의 hidden 출력들을 차례로 수집
        
    output = np.matmul(hidden, pm_output['w']) + pm_output['b'] #마지막 은닉 계층의 출력인 hidden값을 출력 계층으 입력으로 이용
    
    return output, hiddens #역전파용 보조 정보로 전달

#가변적 은닉 계층 구성을 위한 역전파 함수 정의
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens

    hiddens = aux 
    
    #출력 계층에 대한 역전파 처리를 수행한 후에 반복문을 실행해 은닉 계층에 대한 역전파 처리 과정을 은닉층 수만큼 반복
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)
    
    #출력 계층과 은닉 계층의 역전파 처리를 매개
    g_output_hidden = pm_output['w'].transpose() 
    G_hidden = np.matmul(G_output, g_output_hidden)

    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n+1])

        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis=0)
        
        #은닉 계층과 은닉 계층 사이를 매개 
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)

        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['b'] -= LEARNING_RATE * G_b_hid
        

#스위치 함수 정의        
global hidden_config #hidden_config 전역 변수의 설정 여부에 따라 두 부류 함수를 선택적으로 호출 함

def init_model():
    if hidden_config is not None:
        print('은닉 계층 {}개를 갖는 다층 퍼셉트론이 작동되었습니다.'. \
              format(len(hidden_config)))
        init_model_hiddens()
    else:
        print('은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.')
        init_model_hidden1()
    
def forward_neuralnet(x):
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)
    else:
        return forward_neuralnet_hidden1(x)
    
def backprop_neuralnet(G_output, hiddens):
    if hidden_config is not None:
        backprop_neuralnet_hiddens(G_output, hiddens)
    else:
        backprop_neuralnet_hidden1(G_output, hiddens)
        
#은닉 계층 구조 지정 함수 정의
def set_hidden(info):
    global hidden_cnt, hidden_config
    if isinstance(info, int):#매개변수로 주어진 info가 정수인지 판별하여 밑에 알맞은 쪽에 info값 저장
        hidden_cnt = info
        hidden_config = None
    else:
        hidden_config = info